In [21]:
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from sympy import Eq, solve_linear_system, Matrix
from numpy import linalg
import numpy as np
import sympy as sp
import copy

### Python Script based on lecture 'Discrete Event System' of TU Berlin
By: Hutomo Saleh 
#### To use, pip install:
    - sympy
    - numpy

In [22]:
"""
    Functions for Enforceability
"""
def build_Aoc(A, index):
    if isinstance(index, int):
        return np.array(A[:, index])
    if len(index) > 0:
        arr = np.array(A[:, index[0]])
        for i in index[1:]:
            arr = np.vstack((arr, A[:, i]))
        return arr.T
    return 0

def get_index_UU(A, gamma, i_uouc):
    Auc = build_Aoc(A, i_uouc)
    v = np.dot(-gamma, Auc)
    return int((v != 0).nonzero()[0])

def check_ideal_enforcable(A, gamma, i_ouc, i_uouc, x0, b):
    Aouc = build_Aoc(A, i_ouc)
    Auouc = build_Aoc(A, i_uouc)
    
    first = np.dot(-gamma, Aouc)
    print(f"{first} >= {0}")
    second = np.dot(-gamma, Auouc)
    print(f"{second} == {0}")
    third = np.dot(gamma, x0)
    print(f"{third} <= {b}")
    Ac = np.dot(-gamma, A)
    print("Ac:")
    show(Ac)
    
    if np.all(np.dot(-gamma, Aouc) >= 0) and np.all(np.dot(-gamma, Auouc) == 0) and np.all(np.dot(gamma, x0) <= b):
        print("Yes, It is ideally enforceable")
        return True
    else:
        print("It is NOT ideally enforceable")
        return False

def stricter_algorithm(A, gamma, i_ouc, i_uouc=None):
    if gamma.ndim > 1:
        print("Specification must be 1 dimensional. Or run the algorithm for each gamma one by one!")
        return False
    
    """
        Make n by m C Matrix
    """
    n = np.shape(A)[0]
    Auc = build_Aoc(A, i_ouc)
    try:
        if Auc.ndim == 1:
            Auc = Auc.reshape(-1, 1)
    except AttributeError:
        pass

    """
        Observable but unpreventable Transition
    """
    if i_uouc is None:
        I = np.identity(n)
        v = np.dot(gamma, Auc)
        C = np.concatenate((Auc, I, np.zeros((n, 1))), axis=1)
        last_row = np.append(np.concatenate((v, np.zeros(n))), 1)
        C = np.vstack((C, last_row))
        print(f"C:")
        show(C)
        
        while not (np.all(v <= 0)):
            for i, val in enumerate(v):
                if(val > 0):
                    I = []
                    for k, val in enumerate(Auc[:, i]):
                        if(val < 0):
                            I.append(k)
                    if I == []:
                        print("No result")
                        return None
                    print(f"I: {I}")
                    for i_ in I:
                        d = np.lcm(int(abs(Auc[i_, i])), int(abs(v[i])))
                        C_end = d/abs(v[i]) * C[-1, :] + d/abs(Auc[i_, i]) * C[i_, :]
                        spec = C_end[1:-1] + gamma
                        Ac = np.dot(-spec, A)
                        print(f"C last row: {C_end}")
                        print(f"Spec: {spec}")
                        print(f"Ac: {Ac}")
                    if not (np.all(C_end[-(n+1):-1] == 0)):
                        C[-1, :] = C_end
                        v = C_end[:-(n+1)]
                        break
    
    """
        Unobservable and unpreventable Transition
    """
    if i_uouc:
        Auouc = build_Aoc(A, i_uouc)
        try:
            if Auouc.ndim == 1:
                Auouc = Auouc.reshape(-1, 1)
        except AttributeError:
            pass
        I = np.identity(n)
        v = np.dot(gamma, Auc)
        u = np.dot(gamma, Auouc)
        if isinstance(Auc, int):
            C = np.concatenate((Auouc, I, np.zeros((n, 1))), axis=1)
            last_row = np.append(np.concatenate((u, np.zeros(n))), 1)
        else:
            C = np.concatenate((Auc, Auouc, I, np.zeros((n, 1))), axis=1)
            last_row = np.append(np.concatenate((u, v, np.zeros(n))), 1)
        C = np.vstack((C, last_row))
        print(f"C:")
        show(C)
        
        if not isinstance(Auc, int):
            while not (np.all(v <= 0)):
                for i, val in enumerate(v):
                    if(val > 0):
                        I = []
                        for k, val in enumerate(Auc[:, i]):
                            if(val < 0):
                                I.append(k)
                        if I == []:
                            print("No result")
                            return None
                        print(f"I: {I}")
                        for i_ in I:
                            d = np.lcm(int(abs(Auc[i_, i])), int(abs(v[i])))
                            C_end = d/abs(v[i]) * C[-1, :] + d/abs(Auc[i_, i]) * C[i_, :]
                            spec = C_end[1:-1] + gamma
                            Ac = np.dot(-spec, A)
                            print(f"C last row: {C_end}")
                            print(f"Spec: {spec}")
                            print(f"Ac: {Ac}")
                        if not (np.all(C_end[-(n+1):-1] == 0)):
                            C[-1, :] = C_end
                            v = C_end[:-(n+1)]
                            break
        
        while not (np.all(u == 0)):
            for l, val in enumerate(u):
                if(val != 0):
                    Z = []
                    for k, val in enumerate(Auouc[:, l]):
                        if (val != 0) and (np.sign(val) != np.sign(u[l])):
                            Z.append(k)
                    if Z == []:
                        print("No result")
                        return None
                    print(f"Z: {Z}")
                    for z_ in Z:
                        d = np.lcm(int(abs(Auouc[z_, l])), int(abs(u[l])))
                        C_end = d/abs(u[l]) * C[-1, :] + d/abs(Auouc[z_, l]) * C[z_, :]
                        spec = C_end[-(n+1):-1] + gamma
                        Ac = np.dot(-spec, A)
                        print(f"C last row: {C_end}")
                        print(f"Spec: {spec}")
                        print(f"Ac: {Ac}")
                    if not (np.all(C_end[-(n+1):-1] == 0)):
                        C[-1, :] = C_end
                        u = C_end[:-(n+1)]
                        break

In [23]:
"""
    Functions for State Space Equations
"""

# o = sp.symbols('ε')  # ε Epsilon = -inf | Can be changed into any symbol
o = sp.symbols('.')  # For clarity, instead of epsilon

def N(*args):
    """
        Returns n by m N matrix
    """
    if isinstance(args[0], tuple):
        n = args[0][0]
        m = args[0][1]
    else:
        n = args[0]
        m = args[1]
    return np.array([[o for k in range(n)] for i in range(m)])

def E(*args):
    """
        Returns n by m E matrix
    """
    if isinstance(args[0], tuple):
        n = args[0][0]
        m = args[0][1]
    else:
        n = args[0]
        m = args[1]
    arr = N(n, m)
    for i in range(n):
        arr[i, i] = 0
    return arr

def _identical(A, B):
    it = np.nditer(A, flags=['multi_index', "refs_ok"])
    for x in it:
        index = it.multi_index
        if (A[index] != B[index]): return False
    return True

def get_A0_(A, verbose=True):
    k = 1
    Ak = A
    while not _identical(Ak, N(np.shape(A))):
        Ak = mul_matrix(Ak, A)
        k += 1
        if k > np.shape(A)[0]:
            if verbose: print("Cannot compute A0*, there exist a circuit")
            return None
    print(f"K is {k}")
    
    # Calculate A0_
    A0_ = E(np.shape(A))
    Ak = copy.deepcopy(A)
    for i in range(k):
        if i > 0:
            Ak = mul_matrix(Ak, A)
        A0_ = add_matrix(A0_, Ak)
    return A0_

def max_sym(*nums):
    # Max-plus max
    if len(nums) == 1:
        nums = nums[0]
    res = o
    for n in nums:
        if n != o:
            if res == o:
                res = n
            elif (n > res):
                res = n
    return res

def sum_sym(*nums):
    # Max-plus summation
    if len(nums) == 1:
        nums = nums[0]
    res = 0
    for n in nums:
        if n == o:
            return o
        else:
            res += n
    return res
    
def add_matrix(A, B):
    # Max-plus matrix addition
    n = np.shape(A)[0]
    m = np.shape(B)[1]
    arr = sp.Matrix(np.zeros((n, m)))
    for i in range(n):
        for j in range(m):
            arr[i, j] = max_sym(A[i, j], B[i, j])
    return np.array(arr)

def mul_matrix(A, B):
    # Max-plus matrix multiplication
    try:
        n = np.shape(A)[0]
        m = np.shape(B)[1]
        arr = sp.Matrix(np.zeros((n, m)))
        for i in range(n):
            for j in range(m):
                row = A[i, [k for k in range(n)]]
                col = B[[k for k in range(n)], j]
                arr[i, j] = max_sym([sum_sym(row[k], col[k]) for k in range(n)])
    except IndexError:
        arr = B
        it = np.nditer(B, flags=['multi_index', "refs_ok"])
        for x in it:
            index = it.multi_index
            arr[index] = sum_sym(B[index], A)
    return np.array(arr)

def trace(mat):
    n, m = np.shape(mat)
    if not n == m:
        print("Input must be a square matrix")
        return None
    res = max_sym(mat[0, 0], mat[1, 1])
    for i in range(2, n):
        res = max_sym(res, mat[i, i])
    return res

def get_lambda(mat, max_throughput=False):
    """
        returns lambda value using theorem 3.3
        does NOT check for irreducibilty
    """
    n, m = np.shape(mat)
    if not n == m:
        print("Input must be a square matrix")
        return None
    lambda_ = o
    Aj = copy.deepcopy(mat)
    for i in range(n):
        if i >= 1:
            Aj = mul_matrix(Aj, mat)
        lambda_ = max_sym(lambda_, trace(Aj)/(i+1))
    if max_throughput:
        print(f"Max throughput: {1/lambda_}")
    return lambda_

def get_eigen(mat, verbose=False):
    """
        returns lambda and eigenvectors
        of an irreducible matrix
    """
    n, _ = np.shape(mat)
    lambda_ = get_lambda(A)
    Q = mul_matrix(-lambda_, A)
    Q_pow = Q
    Q_star = add_matrix(E(np.shape(A)), Q)
    for i in range(2, n):
        Q_pow = mul_matrix(Q_pow, Q)
        Q_star = add_matrix(Q_star, Q_pow)
    Q_plus = mul_matrix(Q, Q_star)
    if verbose:
        print("\u03BB: ", l)
        print("Q+ = Q x Q*: ")
        _show(Q_plus)
    return Q_plus, lambda_

def get_ss_mats(*matrices):
    a0 = matrices[0]
    a0_ = get_A0_(a0)
    if not a0_: print("Circuit exists!")
    SS = []
    for mat in matrices[1:]:
        arr = mul_matrix(a0_, mat)
        SS.append(arr)
    return SS

def get_trajectory(A, x0, n=5):
    v = mul_matrix(A, x0)
    _show(v)
    for i in range(2, n):
        v = mul_matrix(A, v)
        _show(v)
        
def s(n):
    # Function for printing superscripts
    return "".join(["⁰¹²³⁴⁵⁶⁷⁸⁹"[ord(c)-ord('0')] for c in str(n)]) 

def show(mat):
    if isinstance(mat, list):
        for i in mat:
            show(i)
    else:
        return display(sp.Matrix(mat))

def _show(mat):
    e = sp.symbols('e')
    arr = copy.deepcopy(mat)
    it = np.nditer(mat, flags=['multi_index', "refs_ok"])
    for x in it:
        if x == 0:
            index = it.multi_index
            arr[index] = e
    show(arr)

### Graded Homework

In [15]:
    # HELPER Comment for indexing
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14

    # FROM →
    # TO ↓

"""
    phi 1
"""
A0 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 1
    [6, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, 0, o, o, o, o, o, o, o, 0, o, o, o, o], # 3
    [o, o, 4, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, 0, o, o, o, o, o, o, o, o, o, o], # 5
    [o, o, o, o, 3, o, o, o, o, o, o, o, o, o], # 6
    [o, 0, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, 5, o, o, o, o, o, o, o], # 8
    [o, o, o, o, o, o, o, 0, o, o, o, o, o, 0], # 9
    [o, o, o, o, o, o, o, o, 3, o, o, o, o, o], # 10
    [o, o, o, o, o, 0, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, 4, o, o, o], # 12
    [o, o, o, o, o, o, o, o, o, o, o, 0, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, 3, o]  # 14
    ])

A1 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, 0, o, o, o, o, o, o], # 1
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 3
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, o, o, o, o, o, o, o, o, 0, o, o], # 5
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 6
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 8
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 9
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 10
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 12
    [o, o, o, 0, o, o, o, o, o, o, o, o, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o]  # 14
    ])

In [16]:
print("A0: ")
_show(A0)
print("A1: ")
_show(A1)
A0_ = get_A0_(A0)
print("A0*: ")

A0: 


⎡.  .  .  .  .  .  .  .  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢6  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  e  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  4  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  e  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  3  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  5  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  e  .  .  .  .  .  e⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  3  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  e  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  4  .  .  .⎥
⎢          

A1: 


⎡.  .  .  .  .  .  .  e  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  e  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢          

Cannot compute A0*, there exist a circuit
A0*: 


In [17]:
"""
    phi 2
"""
A0 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, 0, o, o, o, o, o, o], # 1
    [6, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, 0, o, o, o, o, o, o, o, 0, o, o, o, o], # 3
    [o, o, 4, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, 0, o, o, o, o, o, o, o, 0, o, o], # 5
    [o, o, o, o, 3, o, o, o, o, o, o, o, o, o], # 6
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, 5, o, o, o, o, o, o, o], # 8
    [o, o, o, o, o, o, o, 0, o, o, o, o, o, 0], # 9
    [o, o, o, o, o, o, o, o, 3, o, o, o, o, o], # 10
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, 4, o, o, o], # 12
    [o, o, o, o, o, o, o, o, o, o, o, 0, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, 3, o]  # 14
    ])
A1 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 1
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 3
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 5
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 6
    [o, 0, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 8
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 9
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 10
    [o, o, o, o, o, 0, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 12
    [o, o, o, 0, o, o, o, o, o, o, o, o, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o]  # 14
    ])

In [12]:
print("A0: ")
_show(A0)
print("A1: ")
_show(A1)
A0_ = get_A0_(A0)
# print("A0*: ")
xor = '\u2295'
# print("A0*: ")
print(f"A0* = E {xor} A0 {xor} A0{s(2)} {xor} A0{s(3)} {xor} A0{s(4)} {xor} A0{s(5)} {xor} A0{s(6)} {xor} A0{s(7)} {xor} A0{s(8)} {xor} A0{s(9)} : ")
_show(A0_)
G_A = mul_matrix(A0_, A1)
print("A0*A1: ")
_show(G_A)
print("\u03BB : ", get_lambda(G_A, max_throughput=True))

A0: 


⎡.  .  .  .  .  .  .  e  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢6  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  e  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  4  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  e  .  .  .  .  .  .  .  e  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  3  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  5  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  e  .  .  .  .  .  e⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  3  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  4  .  .  .⎥
⎢          

A1: 


⎡.  .  .  .  .  .  .  .  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  e  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢          

K is 10
A0* = E ⊕ A0 ⊕ A0² ⊕ A0³ ⊕ A0⁴ ⊕ A0⁵ ⊕ A0⁶ ⊕ A0⁷ ⊕ A0⁸ ⊕ A0⁹ : 


⎡e   .  .  .  .  .  5   e   .   .  .   .   .   . ⎤
⎢                                                ⎥
⎢6   e  .  .  .  .  11  6   .   .  .   .   .   . ⎥
⎢                                                ⎥
⎢6   e  e  .  .  .  11  6   3   e  10  6   6   3 ⎥
⎢                                                ⎥
⎢10  4  4  e  .  .  15  10  7   4  14  10  10  7 ⎥
⎢                                                ⎥
⎢10  4  4  e  e  .  15  10  7   4  14  10  10  7 ⎥
⎢                                                ⎥
⎢13  7  7  3  3  e  18  13  10  7  17  13  13  10⎥
⎢                                                ⎥
⎢.   .  .  .  .  .  e   .   .   .  .   .   .   . ⎥
⎢                                                ⎥
⎢.   .  .  .  .  .  5   e   .   .  .   .   .   . ⎥
⎢                                                ⎥
⎢.   .  .  .  .  .  5   e   e   .  7   3   3   e ⎥
⎢                                                ⎥
⎢.   .  .  .  .  .  8   3   3   e  10  6   6   3 ⎥
⎢                              

A0*A1: 


⎡.  5   .  .   .  .   .  .  .  .  .  .  .  .⎤
⎢                                           ⎥
⎢.  11  .  .   .  .   .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  11  .  6   .  10  .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  15  .  10  .  14  .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  15  .  10  .  14  .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  18  .  13  .  17  .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  e   .  .   .  .   .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  5   .  .   .  .   .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  5   .  3   .  7   .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  8   .  6   .  10  .  .  .  .  .  .  .  .⎥
⎢                                           ⎥
⎢.  .   .  .   .  e   .  .  .  .  .  .  .  .⎥
⎢                                 

Max throughput: 1/17
λ :  17


In [18]:
"""
    phi 3
"""
A0 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 1
    [6, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, 0, o, o, o, o, o, o, o, o, o, o, o, o], # 3
    [o, o, 4, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, 0, o, o, o, o, o, o, o, 0, o, o], # 5
    [o, o, o, o, 3, o, o, o, o, o, o, o, o, o], # 6
    [o, 0, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, 5, o, o, o, o, o, o, o], # 8
    [o, o, o, 0, o, o, o, 0, o, o, o, o, o, o], # 9
    [o, o, o, o, o, o, o, o, 3, o, o, o, o, o], # 10
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, 4, o, o, o], # 12
    [o, o, o, o, o, o, o, o, o, 0, o, 0, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, 3, o]  # 14
    ])

A1 = np.array([
    #1  2  3  4  5  6  7  8  9 10 11 12 13 14
    [o, o, o, o, o, o, o, 0, o, o, o, o, o, o], # 1
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 2
    [o, o, o, o, o, o, o, o, o, o, o, o, o, 0], # 3
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 4
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 5
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 6
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 7
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 8
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 9
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 10
    [o, o, o, o, o, 0, o, o, o, o, o, o, o, o], # 11
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 12
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o], # 13
    [o, o, o, o, o, o, o, o, o, o, o, o, o, o]  # 14
    ])

In [19]:
print("A0: ")
_show(A0)
print("A1: ")
_show(A1)
A0_ = get_A0_(A0)
xor = '\u2295'
# print("A0*: ")
print(f"A0* = E {xor} A0 {xor} A0{s(2)} {xor} A0{s(3)} {xor} A0{s(4)} {xor} A0{s(5)} {xor} A0{s(6)} {xor} A0{s(7)} : ")
_show(A0_)
G_A = mul_matrix(A0_, A1)
print("A0*A1: ")
_show(G_A)
print("\u03BB : ", get_lambda(G_A, max_throughput=True))

A0: 


⎡.  .  .  .  .  .  .  .  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢6  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  4  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  e  .  .  .  .  .  .  .  e  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  3  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  e  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  5  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  e  .  .  .  e  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  3  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  4  .  .  .⎥
⎢          

A1: 


⎡.  .  .  .  .  .  .  e  .  .  .  .  .  .⎤
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  e⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  e  .  .  .  .  .  .  .  .⎥
⎢                                        ⎥
⎢.  .  .  .  .  .  .  .  .  .  .  .  .  .⎥
⎢          

K is 8
A0* = E ⊕ A0 ⊕ A0² ⊕ A0³ ⊕ A0⁴ ⊕ A0⁵ ⊕ A0⁶ ⊕ A0⁷ : 


⎡e   .   .   .  .  .  .   .  .  .  .  .  .  .⎤
⎢                                            ⎥
⎢6   e   .   .  .  .  .   .  .  .  .  .  .  .⎥
⎢                                            ⎥
⎢6   e   e   .  .  .  .   .  .  .  .  .  .  .⎥
⎢                                            ⎥
⎢10  4   4   e  .  .  .   .  .  .  .  .  .  .⎥
⎢                                            ⎥
⎢10  4   4   e  e  .  .   .  .  .  4  e  .  .⎥
⎢                                            ⎥
⎢13  7   7   3  3  e  .   .  .  .  7  3  .  .⎥
⎢                                            ⎥
⎢6   e   .   .  .  .  e   .  .  .  .  .  .  .⎥
⎢                                            ⎥
⎢11  5   .   .  .  .  5   e  .  .  .  .  .  .⎥
⎢                                            ⎥
⎢11  5   4   e  .  .  5   e  e  .  .  .  .  .⎥
⎢                                            ⎥
⎢14  8   7   3  .  .  8   3  3  e  .  .  .  .⎥
⎢                                            ⎥
⎢.   .   .   .  .  .  .   .  .  .  e  .  .  .⎥
⎢            

A0*A1: 


⎡.  .  .  .  .  .  .  e   .  .  .  .  .  . ⎤
⎢                                          ⎥
⎢.  .  .  .  .  .  .  6   .  .  .  .  .  . ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  6   .  .  .  .  .  e ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  10  .  .  .  .  .  4 ⎥
⎢                                          ⎥
⎢.  .  .  .  .  4  .  10  .  .  .  .  .  4 ⎥
⎢                                          ⎥
⎢.  .  .  .  .  7  .  13  .  .  .  .  .  7 ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  6   .  .  .  .  .  . ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  11  .  .  .  .  .  . ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  11  .  .  .  .  .  4 ⎥
⎢                                          ⎥
⎢.  .  .  .  .  .  .  14  .  .  .  .  .  7 ⎥
⎢                                          ⎥
⎢.  .  .  .  .  e  .  .   .  .  .  .  .  . ⎥
⎢                                          ⎥
⎢.  .  .  

Max throughput: 1/11
λ :  11


### Other example Implementations

In [25]:
gamma = np.array([
    [0, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 0, 0],
])
A = np.array([
    [1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, -1, 0, 0, 0],
    [0, 0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, -1]
])
b = np.array([3, 3, 2])
x0 = np.array([2, 0, 0, 0, 0, 0, 0, 0])
i_ouc = ()
i_uouc = (5, 6, 7)
i = get_index_UU(A, gamma, i_uouc)
check_ideal_enforcable(A, gamma, i_ouc, i_uouc, x0, b)
stricter_algorithm(A, gamma[i], i_ouc, i_uouc)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]] >= 0
[[0 0 0]
 [0 0 0]
 [0 1 0]] == 0
[0 0 0] <= [3 3 2]
Ac:


⎡0  -1  0  1  0   0  0  0  0⎤
⎢                           ⎥
⎢0  0   0  0  -1  0  0  0  1⎥
⎢                           ⎥
⎣0  0   0  0  -1  0  1  0  0⎦

It is NOT ideally enforceable
C:


⎡ 0     0     0    1.0   0    0    0    0    0    0    0    0 ⎤
⎢                                                             ⎥
⎢ 0     0     0     0   1.0   0    0    0    0    0    0    0 ⎥
⎢                                                             ⎥
⎢ 0     0     0     0    0   1.0   0    0    0    0    0    0 ⎥
⎢                                                             ⎥
⎢-1.0   0     0     0    0    0   1.0   0    0    0    0    0 ⎥
⎢                                                             ⎥
⎢-1.0   0     0     0    0    0    0   1.0   0    0    0    0 ⎥
⎢                                                             ⎥
⎢1.0   -1.0   0     0    0    0    0    0   1.0   0    0    0 ⎥
⎢                                                             ⎥
⎢ 0    1.0   -1.0   0    0    0    0    0    0   1.0   0    0 ⎥
⎢                                                             ⎥
⎢ 0     0    1.0    0    0    0    0    0    0    0   1.0   0 ⎥
⎢                                       

Z: [6]
C last row: [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  1.  0.  1.]
Spec: [0. 0. 0. 0. 1. 1. 1. 0.]
Ac: [ 0.  0.  0.  0. -1.  0.  0.  1.  0.]
Z: [7]
C last row: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
Spec: [0. 0. 0. 0. 1. 1. 1. 1.]
Ac: [ 0.  0.  0.  0. -1.  0.  0.  0.  1.]


In [26]:
A = np.array([
    [-1, -1, 0, 0, 1],
    [1, 1, 0, -1, 0],
    [0, 0, -1, 0, 1],
    [0, 0, 1, -1, 0],
    [0, 0, 0, 1, -1],
])
i_ouc = (3)
n = np.shape(A)[0]
Auc = np.array([[1, 1, 2, 1, 1] for i in range(1)]).T
Auc = build_Aoc(A, i_ouc)
if Auc.ndim == 1:
    Auc = Auc.reshape(-1, 1)
print(Auc)
gamma = np.array(
    [0, 0, 0, 0, 1]
)
print(gamma.ndim)
I = np.identity(n)
v = np.dot(gamma, Auc)
print(v)

C = np.concatenate((Auc, I, np.zeros((n, 1))), axis=1)
last_row = np.append(np.concatenate((v, np.zeros(n))), 1)
C = np.vstack((C, last_row))

stricter_algorithm(A, gamma, i_ouc)

[[ 0]
 [-1]
 [ 0]
 [-1]
 [ 1]]
1
[1]
C:


⎡ 0    1.0   0    0    0    0    0 ⎤
⎢                                  ⎥
⎢-1.0   0   1.0   0    0    0    0 ⎥
⎢                                  ⎥
⎢ 0     0    0   1.0   0    0    0 ⎥
⎢                                  ⎥
⎢-1.0   0    0    0   1.0   0    0 ⎥
⎢                                  ⎥
⎢1.0    0    0    0    0   1.0   0 ⎥
⎢                                  ⎥
⎣1.0    0    0    0    0    0   1.0⎦

I: [1, 3]
C last row: [0. 0. 1. 0. 0. 0. 1.]
Spec: [0. 1. 0. 0. 1.]
Ac: [-1. -1.  0.  0.  1.]
C last row: [0. 0. 0. 0. 1. 0. 1.]
Spec: [0. 0. 0. 1. 1.]
Ac: [ 0.  0. -1.  0.  1.]


In [27]:
"""
    Exercise 4.1
"""
gamma = np.array([
    [1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0]
])
A = np.array([
    [1, -1, 0, -1, 0],
    [0, 1, -1, 0, 0],
    [0, -1, 1, -1, 1],
    [0, 0, 0, 1, -1],
    [0, 0, 0, -1, 1],
])
b = np.array([3, 4]).T
x0 = np.array([0, 0, 2, 0, 1]).T
Aouc = np.array((A[:, 2], A[:, 4])).T
Auouc = 0

check_ideal_enforcable(A, gamma, (2, 4), (), x0, b)

[[0 0]
 [1 1]] >= 0
[[0 0 0 0 0]
 [0 0 0 0 0]] == 0
[0 0] <= [3 4]
Ac:


⎡-1  1  0  1  0⎤
⎢              ⎥
⎣-1  0  1  0  1⎦

Yes, It is ideally enforceable


True

In [28]:
"""
    Exercise 4.2
"""
gamma = np.array(
    [0, 0, 0, 0, 1]
)
A = np.array([
    [-1, -1, 0, 0, 1],
    [1, 1, 0, -1, 0],
    [0, 0, -1, 0, 1],
    [0, 0, 1, -1, 0],
    [0, 0, 0, 1, -1],
])
b = np.array([1]).T
x0 = np.array([1, 1, 2, 0, 0]).T
i_ouc = (3)
print("a: ")
check_ideal_enforcable(A, gamma, (), (), x0, b)
print("b: ")
check_ideal_enforcable(A, gamma, (3), (), x0, b)
print("c: ")
stricter_algorithm(A, gamma, i_ouc)

a: 
[0 0 0 0 0] >= 0
[0 0 0 0 0] == 0
0 <= [1]
Ac:


⎡0 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣1 ⎦

Yes, It is ideally enforceable
b: 
-1 >= 0
[0 0 0 0 0] == 0
0 <= [1]
Ac:


⎡0 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣1 ⎦

It is NOT ideally enforceable
c: 
C:


⎡ 0    1.0   0    0    0    0    0 ⎤
⎢                                  ⎥
⎢-1.0   0   1.0   0    0    0    0 ⎥
⎢                                  ⎥
⎢ 0     0    0   1.0   0    0    0 ⎥
⎢                                  ⎥
⎢-1.0   0    0    0   1.0   0    0 ⎥
⎢                                  ⎥
⎢1.0    0    0    0    0   1.0   0 ⎥
⎢                                  ⎥
⎣1.0    0    0    0    0    0   1.0⎦

I: [1, 3]
C last row: [0. 0. 1. 0. 0. 0. 1.]
Spec: [0. 1. 0. 0. 1.]
Ac: [-1. -1.  0.  0.  1.]
C last row: [0. 0. 0. 0. 1. 0. 1.]
Spec: [0. 0. 0. 1. 1.]
Ac: [ 0.  0. -1.  0.  1.]
